### 수질 음용성 여부 데이터
데이터 설명 : 수질 음용성 여부 (Potablillity 컬럼 : 0 ,1 )

---

이상치 제거 후 평가용 데이터로 score를 계산한 결과 0.89 정도의 점수가 나왔다. <br>
이상치 제거는 불필요한 작업이었던 것 같다.

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,0,8.662710,173.531947,20333.079495,5.636388,439.787938,459.633120,16.283311,89.924253,5.120103
1,1,NaN,226.270824,15380.124079,6.661474,NaN,392.558205,14.083110,50.286395,4.516870
2,2,7.583770,217.283262,36343.407055,8.532726,375.964391,393.877683,17.442301,77.722257,3.642289
3,3,6.584813,182.375456,24723.106296,6.238920,NaN,414.350751,17.582615,78.213738,4.404132
4,4,7.179864,180.854211,10859.553752,8.263503,341.302486,358.056264,12.065317,83.329918,3.878447


,ID,Potability
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [2]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2620 entries, 0 to 2619
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2620 non-null   int64  
 1   ph               2225 non-null   float64
 2   Hardness         2620 non-null   float64
 3   Solids           2620 non-null   float64
 4   Chloramines      2620 non-null   float64
 5   Sulfate          2003 non-null   float64
 6   Conductivity     2620 non-null   float64
 7   Organic_carbon   2620 non-null   float64
 8   Trihalomethanes  2488 non-null   float64
 9   Turbidity        2620 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 204.8 KB


In [3]:
train = pd.concat([x_train, y_train['Potability']], axis=1)
train

,ID,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0,8.662710,173.531947,20333.079495,5.636388,439.787938,459.633120,16.283311,89.924253,5.120103,0
1,1,NaN,226.270824,15380.124079,6.661474,NaN,392.558205,14.083110,50.286395,4.516870,1
2,2,7.583770,217.283262,36343.407055,8.532726,375.964391,393.877683,17.442301,77.722257,3.642289,0
3,3,6.584813,182.375456,24723.106296,6.238920,NaN,414.350751,17.582615,78.213738,4.404132,0
4,4,7.179864,180.854211,10859.553752,8.263503,341.302486,358.056264,12.065317,83.329918,3.878447,0
...,...,...,...,...,...,...,...,...,...,...,...
2615,3266,8.218171,116.061950,22465.687557,5.000451,NaN,408.360199,6.732794,90.993268,4.246446,0
2616,3268,NaN,287.370208,41325.443548,5.517280,NaN,375.724519,13.977338,97.859775,4.322325,1
2617,3270,6.503638,163.256634,15000.187769,7.641834,334.103120,517.428392,17.530660,37.765518,4.963184,0
2618,3271,7.243410,188.046296,20877.534841,8.454107,NaN,377.649344,16.357375,71.627136,4.466936,1


In [4]:
train.isnull().sum()

ID                   0
ph                 395
Hardness             0
Solids               0
Chloramines          0
Sulfate            617
Conductivity         0
Organic_carbon       0
Trihalomethanes    132
Turbidity            0
Potability           0
dtype: int64

In [5]:
# ph, Sulfate, Trihalomethanes 결측치 채우기 (x_train, x_test 모두 x_train의 평균으로 채움)

train['ph'].fillna(train['ph'].mean(), inplace=True)
train['Sulfate'].fillna(train['Sulfate'].mean(), inplace=True)
train['Trihalomethanes'].fillna(train['Trihalomethanes'].mean(), inplace=True)

x_test['ph'].fillna(train['ph'].mean(), inplace=True)
x_test['Sulfate'].fillna(train['Sulfate'].mean(), inplace=True)
x_test['Trihalomethanes'].fillna(train['Trihalomethanes'].mean(), inplace=True)

In [6]:
train.isnull().sum()

ID                 0
ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [8]:
# ID 컬럼 제거
train.drop(columns='ID', inplace=True)
x_test_drop = x_test.drop(columns='ID')

In [9]:
# 이상치 제거
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

for i in train.columns:
    train = train[(train[i] <= Q3[i]+1.5*IQR[i]) & (train[i] >= Q1[i]-1.5*IQR[i])]

In [10]:
x_train = train.iloc[:,0:-1]
y = train.iloc[:,-1]

In [11]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(x_train)
x_train_scaled = ss.transform(x_train)
x_test_scaled = ss.transform(x_test_drop)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_train_scaled, y, test_size=0.15, random_state=23)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

rf = RandomForestClassifier(random_state=99)
xgb = XGBClassifier()

rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [14]:
print(accuracy_score(y_train, rf.predict(X_train)))
print(accuracy_score(y_val, rf.predict(X_val)))

print(accuracy_score(y_train, xgb.predict(X_train)))
print(accuracy_score(y_val, xgb.predict(X_val)))

1.0
0.6118012422360248
1.0
0.6055900621118012


In [15]:
pd.DataFrame({'ID':x_test.ID, 'Potability':rf.predict(x_test_scaled)}).to_csv('000000000.csv', index=False)

In [16]:
result = pd.read_csv('000000000.csv')
result

,ID,Potability
0,16,0
1,20,1
2,23,1
3,37,0
4,38,0
...,...,...
651,3267,0
652,3269,0
653,3272,0
654,3273,1


In [17]:
# 평가용 데이터
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_test.csv")
y_test

,ID,Potability
0,16,0
1,20,1
2,23,1
3,37,0
4,38,0
...,...,...
651,3267,0
652,3269,0
653,3272,1
654,3273,1


In [18]:
rf.score(x_test_scaled, y_test['Potability'])

0.8948170731707317